In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import os
import pathlib
import pysam

import warnings
warnings.filterwarnings("ignore")

output_version = "20250324"

# Generate metadata

In [ ]:
maindir = "/media/hieunguyen/HNHD01/raw_data/targetMethyl_analysis"
outputdir = "/media/hieunguyen/HNSD_mini/outdir"

Vi_runs = ["R6801", "R6829Vi", "R6873", "R6914"]
Truong_runs = ["R6782", "R6823", "R6829Truong"]

regions = {
    "Vi": "Vi_amplicons.hg19.bed",
    "Truong": "Truong_amplicons.hg19.bed"
}

regions_hg38 = {
    "Vi": "Vi_amplicons.hg38.bed",
    "Truong": "Truong_amplicons.hg38.bed"
}

mode = "directional"
# mode = "non_directional"

# pic = "Truong"
pic = "Vi"

path_to_main_output = os.path.join(outputdir, "targetMethyl_analysis", output_version, f"{pic}_output", mode)
os.system(f"mkdir -p {path_to_main_output}")


print(f"All output are saved at {path_to_main_output}")


In [ ]:
all_cov_files = [item for item in pathlib.Path(maindir).glob("target_methylation_*/06*/*.cov")]

metadata_dict = {
    "filename"  : [item.name.split(".no_deduplicated")[0] for item in all_cov_files],
    "Run" : [str(item).split("/")[6].replace("_no_dedup", "").replace("target_methylation_", "") for item in all_cov_files],
    "path": [str(item) for item in all_cov_files]
}

metadata = pd.DataFrame.from_dict(metadata_dict, orient="columns")
metadata["mode"] = metadata["Run"].apply(lambda x: "directional" if "_without_non_directional" in x else "non_directional")
metadata["Run"] = metadata["Run"].apply(lambda x: x.split("_")[0])
metadata["PIC"] = metadata["Run"].apply(lambda x: "Vi" if x in Vi_runs else "Truong")
metadata["bam_path"] = metadata["path"].apply(lambda x: str(x).replace("06_methylation_extract", "05_sorted_bam").replace(".bedGraph.gz.bismark.zero.cov", ".sorted.bam"))
metadata = metadata[(metadata["mode"] == mode) & (metadata["PIC"] == pic)]

##### regiondf for hg19
regiondf = pd.read_csv(regions[pic], sep = "\t", header = None)
regiondf.columns = ["chrom", "start", "end"]
regiondf = regiondf[["chrom", "start", "end"]]
regiondf["region_name"] = regiondf[["chrom", "start", "end"]].apply(
    lambda x: f"region_{x[0]}_{x[1]}_{x[2]}", axis = 1
)
regiondf["bam_region"] = regiondf[["chrom", "start", "end"]].apply(
    lambda x: f"{x[0].replace('chr', '')}:{x[1]}-{x[2]}", axis = 1
)

##### regiondf for hg38
regiondf_hg38 = pd.read_csv(regions_hg38[pic], sep = ",", header = None)
regiondf_hg38.columns = ["chrom", "start", "end"]
regiondf_hg38 = regiondf_hg38[["chrom", "start", "end"]]
regiondf_hg38["region_name"] = regiondf_hg38[["chrom", "start", "end"]].apply(
    lambda x: f"region_{x[0]}_{x[1]}_{x[2]}", axis = 1
)
regiondf_hg38["bam_region"] = regiondf_hg38[["chrom", "start", "end"]].apply(
    lambda x: f"{x[0].replace('chr', '')}:{x[1]}-{x[2]}", axis = 1
)

metadata

# Count on/off target reads

In [ ]:
metadata["num_total_reads"] = metadata["bam_path"].apply(lambda x: int(pysam.samtools.view("-c", x, catch_stdout=True)))

def count_read_in_region(bam_path, region, chr_mode = False):
    all_reads = []
    bamfile = pysam.AlignmentFile(bam_path, "rb")
    if chr_mode:
        region = f"chr{region}"
    fetched_obj = bamfile.fetch(region = region)
    for read in fetched_obj:
        all_reads.append(read)
    return(len(all_reads))

for region_name in regiondf.region_name.unique():
    bam_region = regiondf[regiondf["region_name"] == region_name]["bam_region"].values[0]
    metadata[region_name] = metadata["bam_path"].apply(lambda x: count_read_in_region(x, bam_region))
    metadata[f"pct_{region_name}"] = metadata[region_name] / metadata["num_total_reads"] * 100
    
metadata.to_excel(os.path.join(path_to_main_output, f"{pic}_read_count_in_region.xlsx"), index = False)


# Process cov file

In [ ]:
metadata.head()

In [ ]:
regiondf.head()

In [ ]:
all_covdf = dict()
for run in metadata.Run.unique():
      for filename in metadata[metadata["Run"] == run]["filename"].unique():
            path_to_save_cov = os.path.join(path_to_main_output, run, filename)
            os.system(f"mkdir -p {path_to_save_cov}")

            covdf = pd.read_csv(metadata[metadata["filename"] == filename]["path"].values[0], header = None, sep = "\t")
            covdf.columns = ["chrom", "start", "end", "meth_density", "countC", "countT"]
            all_covdf[filename] = covdf
            for region in regiondf.region_name.unique():
                  chrom = int(region.split("_")[1].replace("chr", ""))
                  start = int(region.split("_")[2])
                  end = int(region.split("_")[3])

                  covdf[(covdf["chrom"] == chrom) & 
                        (covdf["start"] >= start) & (covdf["start"] <= end)].to_excel(f"{path_to_save_cov}/{region}.xlsx", index = False)


# Check sample: CONTROL114, R6447

## Check number of reads

In [ ]:
control114_cov = "/media/hieunguyen/HNHD01/raw_data/targetMethyl_analysis/CONTROL114/65-CONTROL114CT836T_M550-M750.cg_cov.bed"
control114_bam = "/media/hieunguyen/HNHD01/raw_data/targetMethyl_analysis/CONTROL114/65-CONTROL114CT836T_M550-M750.deduplicated.sort.bam"
control114_cov_hg19 = "/media/hieunguyen/HNHD01/raw_data/targetMethyl_analysis/CONTROL114/65-CONTROL114CT836T_M550-M750.cg_cov.hg19.bed"


path_to_save_cov = os.path.join(path_to_main_output, "R6447", "CONTROL114")
os.system(f"mkdir -p {path_to_save_cov}")
os.system(f"mkdir -p {path_to_save_cov}/hg38")
os.system(f"mkdir -p {path_to_save_cov}/hg19")

covdf = pd.read_csv(control114_cov, header = None, sep = "\t")
covdf.columns = ["chrom", "start", "end", "meth_density", "countC", "countT"]

covdfhg19 = pd.read_csv(control114_cov_hg19, header = None, sep = "\t")
covdfhg19.columns = ["chrom", "start", "end", "meth_density", "countC", "countT"]
covdfhg19["countC"] = covdf["countC"].values
covdfhg19["countT"] = covdf["countT"].values

for region in regiondf_hg38.region_name.unique():
        chrom = region.split("_")[1]
        start = int(region.split("_")[2])
        end = int(region.split("_")[3])

        covdf[(covdf["chrom"] == chrom) & 
            (covdf["start"] >= start) & (covdf["start"] <= end)].to_excel(f"{path_to_save_cov}/hg38/{region}.xlsx", index = False)
        
        print(covdf[(covdf["chrom"] == chrom) & 
            (covdf["start"] >= start) & (covdf["start"] <= end)].shape)
        
for region in regiondf.region_name.unique():
        chrom = region.split("_")[1]
        start = int(region.split("_")[2])
        end = int(region.split("_")[3])

        covdfhg19[(covdfhg19["chrom"] == chrom) & 
            (covdfhg19["start"] >= start) & (covdfhg19["start"] <= end)].to_excel(f"{path_to_save_cov}/hg19/{region}.xlsx", index = False)
        
        print(covdfhg19[(covdfhg19["chrom"] == chrom) & 
            (covdfhg19["start"] >= start) & (covdfhg19["start"] <= end)].shape)
        
all_covdf["CONTROL114"] = covdfhg19

In [ ]:
metadata_CONTROL114 = pd.DataFrame.from_dict({
    "filename": ["CONTROL114"],
    "Run": ["R6447"],
    "bam_path": [control114_bam],
    "path": [control114_cov]
})

metadata_CONTROL114["num_total_reads"] = metadata_CONTROL114["bam_path"].apply(lambda x: int(pysam.samtools.view("-c", x, catch_stdout=True)))

for region_name in regiondf_hg38.region_name.unique():
    bam_region = regiondf_hg38[regiondf_hg38["region_name"] == region_name]["bam_region"].values[0]
    metadata_CONTROL114[region_name] = metadata_CONTROL114["bam_path"].apply(lambda x: count_read_in_region(x, bam_region, chr_mode = True))
    metadata_CONTROL114[f"pct_{region_name}"] = metadata_CONTROL114[region_name] / metadata_CONTROL114["num_total_reads"] * 100

metadata_CONTROL114.to_excel(os.path.join(path_to_main_output, f"CONTROL114_read_count_in_region.xlsx"), index = False)

## Compare methylation density between 2 samples

In [ ]:
mean_methyldf = pd.DataFrame(data = regiondf.region_name.unique(), columns = ["region"])

def get_mean_methyl_in_region(region, filename):

    if filename == "CONTROL114":
        df = pd.read_excel(f"{os.path.join(path_to_main_output, 'R6447', 'CONTROL114')}/hg19/{region}.xlsx")
        run = "R6447"
    else:
        run = metadata[metadata["filename"] == filename]["Run"].values[0]
        df = pd.read_excel(f"{os.path.join(path_to_main_output, run, filename)}/{region}.xlsx")
    if df.shape[0] == 0:
        return "no data available"
    else:
        mean_methyl = df.meth_density.mean()
        return mean_methyl

for filename in all_covdf.keys():
    mean_methyldf[filename] = mean_methyldf["region"].apply(lambda x: get_mean_methyl_in_region(x, filename))

mean_methyldf.to_excel(os.path.join(path_to_main_output, f"{pic}_mean_methyl_in_region_compare_CONTROL114.xlsx"), index = False)


In [ ]:
metadata